# 交叉熵
在信息论中，交叉熵衡量的是当你有一个估计的概率分布 q，并用这个分布来编码信息时，
平均需要多少位（比特）来正确地识别一个事件。简而言之，交叉熵描述了使用非最佳概率分布进行编码时，传达事件所需的额外信息量。交叉熵函数和负对数似然形式等价。
由于y是一个 长度为q的独热编码向量，所以除了一个项以外的所有项j都消失了。由于所有$y_j$都是预测的概率，所以它们的对数永远不会大于0 <br>


补充，熵的公式、相对熵（KL散度）公式，以及它们与交叉熵的关系。<br>
熵的公式(熵就是平均信息量,信息量的公式为$I(x) = - \log p(x)$)：<br>
$H(p) = - \sum_{i=1}^{N} p(x_i) \log p(x_i)$ <br>
相对熵（KL散度）公式：<br>
$D_{KL}(P\| Q) = \sum_{i=1}^{N} p(x_i) \log\left(\frac{p(x_i)}{q(x_i)}\right)$<br>
熵、交叉熵和KL散度的关系——交叉熵可以分解为熵和KL散度的和：<br>
$H(P,Q) = H(P)+D_{KL}(P\| Q)$<br>
交叉熵的公式：<br>
$H(P, Q) = - \sum_{i=1}^{N} p(x_i) \log q(x_i)$ <br>
其中：<br>
- $p(x_i)$是真实分布P中事件$x_i$发生的概率<br>
- $q(x_i)$是我们用近似分布Q对事件$x_i$的预测概率。<br>

这个公式表示，在真实分布下下事件发生的频率为$p(x_i)$时，我们用分布Q来编码这些事件所需的平均信息量。直观地说，如果我们一直用Q来编码或预测P中的事件，我们每次会“付出”多少信息代价(“代价”指的是我们需要消耗多少比特或多少单位的信息来正确地编码或预测某个事件。)。<br>
简化为分类问题中使用独热编码的形式<br>
$H(P, Q) = - \sum_{i=1}^{N} y_i \log \hat{y}_i$

### 调用CrossEntropyLoss标准库

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

loss = nn.CrossEntropyLoss(weight=torch.tensor([1,2,3,5]).float(),ignore_index=-100,reduction="mean")
_inputs = torch.arange(32).float().reshape(2, 4, 4) #2个批次，每个批次 4个样本，然后维度（输出类别个数）为4
_targets = torch.tensor([[1, 2, 3, 3], [2, 3, 1, 0]], dtype=torch.long) # 2个批次，每个批次4个样本对应输出的target的index
output = loss(_inputs.view(-1,4), _targets.view(-1))
print(_inputs.view(-1,4), _targets.view(-1))
print(_inputs)
print(_targets)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.],
        [16., 17., 18., 19.],
        [20., 21., 22., 23.],
        [24., 25., 26., 27.],
        [28., 29., 30., 31.]]) tensor([1, 2, 3, 3, 2, 3, 1, 0])
tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]],

        [[16., 17., 18., 19.],
         [20., 21., 22., 23.],
         [24., 25., 26., 27.],
         [28., 29., 30., 31.]]])
tensor([[1, 2, 3, 3],
        [2, 3, 1, 0]])


### 手动实现交叉熵

In [5]:
class CrossEntropy(nn.Module):
    def __init__(self, weights = None, reduction = "mean", ignore_index = -100) -> None:
        super().__init__()
        self.weights = weights
        self.reduction = reduction
        self.ignore_index = ignore_index

    def forward(self, input, target):
        #ignore_index
        print(target)
        index_target = torch.where(target != self.ignore_index)[0] #返回target中值不为ignore_index的索引

        print(f"media:{index_target}")
        # target = torch.gather(target, 0, index_target)
        target = target[index_target]

        print(f"target{target}")
        input = input[index_target]
        print(input.shape)

        print(F.one_hot(target,input.shape[-1])) #input.shape[-1]为label的长度
        #计算每一个样本的loss
        f_loss = -torch.mul(F.one_hot(target,input.shape[-1]), torch.log_softmax(input, dim=1)) 
        # f_loss = -F.log_softmax(input, dim=1)[target] 之后看看
        #相同的作用
        # inputs_log_softmax = torch.log_softmax(inputs, dim=-1)
        # log_softmax_prob = torch.gather(inputs_log_softmax, dim=-1, index=targets.view(-1, 1))
        each_batch_loss = torch.sum(f_loss, dim=-1)
        # print("###")
        # print(each_batch_loss)
        # print(each_batch_loss.shape)

        #Weights
        if self.weights is not None:
            # print(target)
            # print("!!!")
            # print(self.weights)
            self.weights = self.weights[target]
            # print(self.weights)
            each_batch_loss = torch.mul(self.weights, each_batch_loss)
            if self.reduction == "mean":
                # print(self.weights)
                # print(torch.sum(self.weights) )
                loss =  torch.sum(each_batch_loss)/torch.sum(self.weights) 
            elif self.reduction == "sum":
                loss =  torch.sum(each_batch_loss)
            else:
                return each_batch_loss

        #计算总loss
        else:
            if self.reduction == "mean":
                loss = torch.mean(each_batch_loss)
            elif self.reduction == "sum":
                loss = torch.sum(each_batch_loss)
            else:
                return each_batch_loss
        
        return loss

loss2 = CrossEntropy(weights=torch.tensor([1,2,3,5]).float(),ignore_index=1,reduction="mean")
output2 = loss2(_inputs.view(-1,4), _targets.view(-1))
# print(output2)

tensor([1, 2, 3, 3, 2, 3, 1, 0])
media:tensor([1, 2, 3, 4, 5, 7])
targettensor([2, 3, 3, 2, 3, 0])
torch.Size([6, 4])
tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0]])
tensor([2, 3, 3, 2, 3, 0])
!!!
tensor([1., 2., 3., 5.])
tensor([3., 5., 5., 3., 5., 1.])
